# GA Data Science 10 (DAT10) - Lab 14

### Recommendation Systems

Francesco Mosconi, Justin Breucop

### Today

1. Simple similarity based recommendation system
2. Recsys

## Similarity based Recommendation System: Beers


Let's build a recommendation system to recommend types of beers based on user reviews

Usual imports (numpy, pandas)

In [2]:
import pandas as pd
import numpy as np

First of all let's get the data

Curl lets you pull data

In [3]:
! curl -O https://s3.amazonaws.com/demo-datasets/beer_reviews.tar.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (60) SSL certificate problem: unable to get local issuer certificate
More details here: http://curl.haxx.se/docs/sslcerts.html

curl performs SSL certificate verification by default, using a "bundle"
 of Certificate Authority (CA) public keys (CA certs). If the default
 bundle file isn't adequate, you can specify an alternate file
 using the --cacert option.
If this HTTPS server uses a certificate signed by a CA represented in
 the bundle, the certificate verification probably failed due to a
 problem with the certificate (it might be expired, or the name might
 not match the domain name in the URL).
If you'd like to turn off curl's verification of the certificate, use
 the -k (or --insecure) option.


Import data in a pandas dataframe called "allbeers". Use the compression keyword

In [4]:
allbeers = pd.read_csv("beer_reviews.tar.gz", compression='gzip')

c:\Users\Admin\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


Let's look at the data

In [5]:
allbeers.head()

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


Let's restrict this to the top 250 beers. Use the value_counts() method select the top 250 beers.
Assign the selected beers to a dataset called df

'# we use top 250 because we have the most data, so we can make recommendations.

In [6]:
n = 250
top_n = allbeers.beer_name.value_counts().index[:n]
df = allbeers[allbeers.beer_name.isin(top_n)]
df.head()

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
798,1075,Caldera Brewing Company,1212201268,4.5,4.5,4,grumpy,American Double / Imperial Stout,4.0,4.5,Imperial Stout,NaN,42964
1559,11715,Destiny Brewing Company,1137124057,4.0,3.5,4,blitheringidiot,American Pale Ale (APA),3.5,3.5,Pale Ale,4.5,26420
1560,11715,Destiny Brewing Company,1129504403,4.0,2.5,4,NeroFiddled,American Pale Ale (APA),4.0,3.5,Pale Ale,4.5,26420
1563,11715,Destiny Brewing Company,1137125989,3.5,3.0,4,blitheringidiot,American IPA,4.0,4.0,IPA,NaN,26132
1564,11715,Destiny Brewing Company,1130936611,3.0,3.0,3,Gavage,American IPA,4.0,3.5,IPA,NaN,26132


How big is this dataset?

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355275 entries, 798 to 1586564
Data columns (total 13 columns):
beer_reviews/         355275 non-null object
brewery_name          355275 non-null object
review_time           355275 non-null float64
review_overall        355275 non-null float64
review_aroma          355275 non-null float64
review_appearance     355275 non-null float64
review_profilename    355175 non-null object
beer_style            355275 non-null object
review_palate         355275 non-null float64
review_taste          355275 non-null float64
beer_name             355275 non-null object
beer_abv              353477 non-null float64
beer_beerid           355275 non-null float64
dtypes: float64(8), object(5)
memory usage: 37.9+ MB


### Pivot Table

Aggregate the data in a pivot table using the pivot_table method. Display the mean review_overall for each beer_name aggregating the review_overall values by review_profilename. Use the mean as aggregator.

In [9]:
df_pivot = pd.pivot_table(df, values=["review_overall"],
        columns=["beer_name", "review_profilename"],
        aggfunc=np.mean)
#pivot_table converts to a multi-index series. Unstack converts to a dataframe where the last index becomes our column head
df_wide = df_pivot.unstack(-1)
df_wide

# unstack changes the rows to the column names
# lots of nulls - this makes sense because not everyone has drank every beer and reviewed it. Column headers = users
# illustration of a sparse dataset
# index -1 - takes the last one 

review_profilename                                                 0110x011  \
               beer_name                                                      
review_overall #9                                                       NaN   
               120 Minute IPA                                           NaN   
               1554 Enlightened Black Ale                               NaN   
               60 Minute IPA                                            NaN   
               90 Minute IPA                                            5.0   
               Aecht Schlenkerla Rauchbier Märzen                       NaN   
               AleSmith IPA                                             5.0   
               AleSmith Speedway Stout                                  4.5   
               Allagash White                                           NaN   
               Alpha King Pale Ale                                      5.0   
               Anchor Liberty Ale                                       NaN   
               Anchor Old Foghorn                                       NaN   
               Anchor Porter                                            NaN   
               Anchor Steam Beer                                        NaN   
               ApriHop                                                  NaN   
               Arrogant Bastard Ale                                     NaN   
               Ayinger Celebrator Doppelbock                            NaN   
               Bass Pale Ale                                            NaN   
               Bell's Expedition Stout                                  4.0   
               Bell's Hopslam Ale                                       4.5   
               Bell's Java Stout                                        NaN   
               Bell's Kalamazoo Stout                                   NaN   
               Bell's Oberon Ale                                        NaN   
               Bell's Special Double Cream Stout                        NaN   
               Big Bear Black Stout                                     NaN   
               Black Butte Porter                                       NaN   
               Blanche De Chambly                                       NaN   
               Blind Pig IPA                                            4.5   
               Blue Moon Belgian White                                  NaN   
               Boddingtons Pub Ale                                      NaN   
...                                                                     ...   
               Stone Smoked Porter                                      NaN   
               Stone Sublimely Self-Righteous Ale                       5.0   
               Storm King Stout                                         4.0   
               Supplication                                             4.0   
               Ten FIDY                                                 4.0   
               Terrapin Coffee Oatmeal Imperial Stout                   NaN   
               The Abyss                                                4.5   
               Three Philosophers Belgian Style Blend (Quadrupel)       NaN   
               Titan IPA                                                NaN   
               Trappist Westvleteren 12                                 4.5   
               Trappistes Rochefort 10                                  NaN   
               Trappistes Rochefort 8                                   NaN   
               Tripel Karmeliet                                         NaN   
               Troegenator Double Bock                                  NaN   
               Trois Pistoles                                           NaN   
               Tröegs Hopback Amber Ale                                 NaN   
               Tröegs Mad Elf                                           4.5   
               Tröegs Nugget Nectar               

In [16]:
# df_pivot

Display the head of the pivot table, but only for 5 users (columns are users)

In [17]:
df_wide.ix[0:5, 0:5]

review_profilename                         0110x011  02maxima  03SVTCobra  \
               beer_name                                                    
review_overall #9                               NaN       NaN         NaN   
               120 Minute IPA                   NaN       NaN         NaN   
               1554 Enlightened Black Ale       NaN       NaN         NaN   
               60 Minute IPA                    NaN       NaN         NaN   
               90 Minute IPA                      5       NaN         NaN   

review_profilename                         05Harley  0Naught0  
               beer_name                                       
review_overall #9                               NaN       NaN  
               120 Minute IPA                     4       NaN  
               1554 Enlightened Black Ale       NaN       NaN  
               60 Minute IPA                    NaN       NaN  
               90 Minute IPA                      4       NaN

### Discussion: what do you notice in this table?

#### Data munging
Set Nans to zero

In [18]:
df_wide = df_wide.fillna(0)

Check that columns are users

In [19]:
df_wide.columns[:10]

Index([u'0110x011', u'02maxima', u'03SVTCobra', u'05Harley', u'0Naught0',
       u'0beerguy0', u'0runkp0s', u'0tt0', u'1000Bottles', u'1001111.0'],
      dtype='object', name=u'review_profilename')

Check that rows are beers

In [21]:
df_wide.index.levels[0]
beer_names = df_wide.index.levels[1]
beer_names

Index([u'#9', u'120 Minute IPA', u'1554 Enlightened Black Ale',
       u'60 Minute IPA', u'90 Minute IPA',
       u'Aecht Schlenkerla Rauchbier Märzen', u'AleSmith IPA',
       u'AleSmith Speedway Stout', u'Allagash White', u'Alpha King Pale Ale', 
       ...
       u'Vanilla Porter', u'Weihenstephaner Hefeweissbier',
       u'Weihenstephaner Korbinian', u'Westmalle Trappist Dubbel',
       u'Westmalle Trappist Tripel', u'World Wide Stout',
       u'Yeti Imperial Stout', u'Young's Double Chocolate Stout',
       u'Yuengling Traditional Lager', u'Éphémère (Apple)'],
      dtype='object', name=u'beer_name', length=250)

### Calculate distance between beers

We're going to use cosine_similarity from scikit-learn to compute the distance between all beers

Imports

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
# calculates cosine similarity of vectors
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

Apply cosine similarity to df_wide to calculate pairwise distances

In [23]:
dists = cosine_similarity(df_wide)
dists
# 0 to 1. 1 is perfectly similar. 0 - not at all

array([[ 1.        ,  0.27540494,  0.27410345, ...,  0.32928048,
         0.34805798,  0.31249922],
       [ 0.27540494,  1.        ,  0.25151873, ...,  0.2854835 ,
         0.23301356,  0.2802485 ],
       [ 0.27410345,  0.25151873,  1.        , ...,  0.31629515,
         0.22521858,  0.2737628 ],
       ..., 
       [ 0.32928048,  0.2854835 ,  0.31629515, ...,  1.        ,
         0.28025764,  0.34504013],
       [ 0.34805798,  0.23301356,  0.22521858, ...,  0.28025764,
         1.        ,  0.25526913],
       [ 0.31249922,  0.2802485 ,  0.2737628 , ...,  0.34504013,
         0.25526913,  1.        ]])

In [25]:
dists.shape
# square matrix of 250 beers w cosine similarity of each beer to every other beer. 

(250L, 250L)

### Discussion: what type of object is dists?

Convert dists to a Pandas DataFrame, use the index as column index as well (distances are a square matrix).

In [45]:
dists = pd.DataFrame(dists)
dists.columns = beer_names
dists.index = beer_names
dists.ix[0:10, 0:10]

dists
# higher value means closer in relation

beer_name,#9,120 Minute IPA,1554 Enlightened Black Ale,60 Minute IPA,90 Minute IPA,Aecht Schlenkerla Rauchbier Märzen,AleSmith IPA,AleSmith Speedway Stout,Allagash White,Alpha King Pale Ale,...,Vanilla Porter,Weihenstephaner Hefeweissbier,Weihenstephaner Korbinian,Westmalle Trappist Dubbel,Westmalle Trappist Tripel,World Wide Stout,Yeti Imperial Stout,Young's Double Chocolate Stout,Yuengling Traditional Lager,Éphémère (Apple)
beer_name,,,,,,,,,,,,,,,,,,,,,
#9,1.000000,0.275405,0.274103,0.388364,0.365175,0.253841,0.228479,0.227612,0.340681,0.293315,...,0.266570,0.312395,0.276463,0.233554,0.276763,0.286534,0.299032,0.329280,0.348058,0.312499
120 Minute IPA,0.275405,1.000000,0.251519,0.378258,0.410366,0.262425,0.315971,0.337541,0.282273,0.336796,...,0.201428,0.312193,0.282320,0.270800,0.301144,0.418214,0.337978,0.285483,0.233014,0.280248
1554 Enlightened Black Ale,0.274103,0.251519,1.000000,0.319887,0.314028,0.252486,0.266866,0.261761,0.260275,0.307296,...,0.285846,0.300474,0.292369,0.265445,0.271656,0.262771,0.295029,0.316295,0.225219,0.273763
60 Minute IPA,0.388364,0.378258,0.319887,1.000000,0.533042,0.316928,0.312343,0.307627,0.360975,0.385249,...,0.285143,0.413405,0.329941,0.308774,0.355926,0.358224,0.391041,0.399840,0.326916,0.339324
90 Minute IPA,0.365175,0.410366,0.314028,0.533042,1.000000,0.312861,0.344218,0.358754,0.356804,0.418582,...,0.262775,0.436398,0.343738,0.333099,0.387312,0.405116,0.414385,0.395031,0.301877,0.332292
Aecht Schlenkerla Rauchbier Märzen,0.253841,0.262425,0.252486,0.316928,0.312861,1.000000,0.244490,0.246063,0.297672,0.263248,...,0.212733,0.350730,0.352677,0.326778,0.334759,0.296632,0.299658,0.331671,0.218881,0.310884
AleSmith IPA,0.228479,0.315971,0.266866,0.312343,0.344218,0.244490,1.000000,0.521889,0.277409,0.400741,...,0.196116,0.363533,0.292208,0.270008,0.297241,0.311192,0.365862,0.262667,0.192580,0.246690
AleSmith Speedway Stout,0.227612,0.337541,0.261761,0.307627,0.358754,0.246063,0.521889,1.000000,0.273930,0.420247,...,0.208057,0.372083,0.304775,0.284294,0.321791,0.361354,0.393764,0.277564,0.199115,0.263719
Allagash White,0.340681,0.282273,0.260275,0.360975,0.356804,0.297672,0.277409,0.273930,1.000000,0.295666,...,0.216042,0.373509,0.321242,0.290985,0.323670,0.299052,0.317692,0.338660,0.283443,0.343491


Select some beers and look their distances to other beers

In [46]:
beers_i_like = ['Sierra Nevada Pale Ale', '120 Minute IPA', 'Allagash White']
dists[beers_i_like].head()
# now have cosine similarity to 250 other beers

beer_name,Sierra Nevada Pale Ale,120 Minute IPA,Allagash White
beer_name,,,
#9,0.373968,0.275405,0.340681
120 Minute IPA,0.301693,1.000000,0.282273
1554 Enlightened Black Ale,0.330033,0.251519,0.260275
60 Minute IPA,0.459641,0.378258,0.360975
90 Minute IPA,0.441189,0.410366,0.356804


Sum the distances of my favourite beers by row, to have one distance from each beer in the sample

In [48]:
beers_summed = dists[beers_i_like].apply(lambda row: np.sum(row), axis=1)
beers_summed = np.sum(dists[beers_i_like], axis=1)
beers_summed

beer_name
#9                                                    0.990054
120 Minute IPA                                        1.583966
1554 Enlightened Black Ale                            0.841827
60 Minute IPA                                         1.198874
90 Minute IPA                                         1.208359
Aecht Schlenkerla Rauchbier Märzen                    0.853397
AleSmith IPA                                          0.887446
AleSmith Speedway Stout                               0.881959
Allagash White                                        1.634784
Alpha King Pale Ale                                   0.976927
Anchor Liberty Ale                                    1.064927
Anchor Old Foghorn                                    0.944878
Anchor Porter                                         0.999485
Anchor Steam Beer                                     1.101660
ApriHop                                               1.089122
Arrogant Bastard Ale                         

#### Performance

Optional: which one is faster? use ```%timeit``` to check

-  only one timeit bc only testing one line for how long it takes 
- doesn't matter if have best recommender - if takes a long time, user will prob leave page before then
- np is much faster (see 2nd row below). when start working at scale, need to consider packages you use. 

In [38]:
%timeit dists[beers_i_like].apply(lambda row: np.sum(row), axis=1)

100 loops, best of 3: 8.98 ms per loop


In [39]:
%timeit np.sum(dists[beers_i_like], axis=1)

1000 loops, best of 3: 461 µs per loop


#### Ranking

Sort summed beers from best to worse

In [49]:
beers_summed = beers_summed.order(ascending=False)
beers_summed
# numbers are valuable in relation to one another

beer_name
Sierra Nevada Pale Ale                        1.654205
Allagash White                                1.634784
120 Minute IPA                                1.583966
HopDevil Ale                                  1.224217
Sierra Nevada Celebration Ale                 1.215156
90 Minute IPA                                 1.208359
60 Minute IPA                                 1.198874
Stone Ruination IPA                           1.194210
Stone IPA (India Pale Ale)                    1.193193
Storm King Stout                              1.192405
Arrogant Bastard Ale                          1.189981
Sierra Nevada Bigfoot Barleywine Style Ale    1.178245
Prima Pils                                    1.178093
Brooklyn Black Chocolate Stout                1.156365
Ayinger Celebrator Doppelbock                 1.148356
Hennepin (Farmhouse Saison)                   1.147501
Samuel Adams Boston Lager                     1.146304
Hop Rod Rye                                   1.140271


Filter out the beers used as input and transform to list

In [51]:
ranked_beers = beers_summed.index[beers_summed.index.isin(beers_i_like)==False]
ranked_beers = ranked_beers.tolist()
ranked_beers[:5]

['HopDevil Ale',
 'Sierra Nevada Celebration Ale',
 '90 Minute IPA',
 '60 Minute IPA',
 'Stone Ruination IPA']

In [44]:
ranked_beers[-6:-1]

["Leinenkugel's Sunset Wheat",
 'Bud Light',
 'Corona Extra',
 'Miller High Life',
 'Coors Light']

###Pair Programming!

Define a function that does what we just did for an arbitrary input list of beers. it should also receive the maximum number of beers requested n as optional parameter.

In [77]:
def get_beer(beer_choices, x):
    if type(beer_choices) != list:
        return "Please enter list:" 
    else: 
        beers_summed = dists[beer_choices].apply(lambda row: np.sum(row), axis=1)
        ranked_beers = beers_summed.index[beers_summed.index.isin(beer_choices)==False]
        ranked_beers = ranked_beers.tolist()
        return ranked_beers[:x]
    
get_beer(['120 Minute IPA','Coors Light'], 6)

['#9',
 '1554 Enlightened Black Ale',
 '60 Minute IPA',
 '90 Minute IPA',
 'Aecht Schlenkerla Rauchbier M\xc3\xa4rzen',
 'AleSmith IPA']

In [72]:
type(beers_i_like)

list

Test your function. Find the 10 beers most similar to "120 Minute IPA"

In [62]:
get_beer('120 Minute IPA', 10)

['#9',
 '1554 Enlightened Black Ale',
 '60 Minute IPA',
 '90 Minute IPA',
 'Aecht Schlenkerla Rauchbier M\xc3\xa4rzen',
 'AleSmith IPA',
 'AleSmith Speedway Stout',
 'Allagash White',
 'Alpha King Pale Ale',
 'Anchor Liberty Ale']

Try again with the 10 beers most similar to ["Coors Light", "Bud Light", "Amstel Light"]

In [79]:
get_beer(['Coors Light', 'Bud Light'], 10)

['#9',
 '120 Minute IPA',
 '1554 Enlightened Black Ale',
 '60 Minute IPA',
 '90 Minute IPA',
 'Aecht Schlenkerla Rauchbier M\xc3\xa4rzen',
 'AleSmith IPA',
 'AleSmith Speedway Stout',
 'Allagash White',
 'Alpha King Pale Ale']

Optional: register an account on yhat and deploy your model following the instructions [here](https://docs.yhathq.com/python/examples/beer-recommender) and [here](http://nbviewer.ipython.org/gist/glamp/20a18d52c539b87de2af)

## Recsys

A python library for implementing a recommender system. If you'd like to, I recommend you explore this project. It's an efficient way to get a recommendation engine off the ground. The example below uses SVD.

In [ ]:
"""
##install python-recsys

### first install dependencies

pip install csc-pysparse networkx divisi2

### then install recsys
git clone https://github.com/python-recsys/python-recsys.git
cd python-recsys/

python setup.py install
"""

Load recsys.algotihm, set VERBOSE = True import SVD class

In [1]:
import recsys.algorithm
recsys.algorithm.VERBOSE = True
from recsys.algorithm.factorize import SVD

Let's look at the files

In [ ]:
! ls movielens

Import 'movies.dat' to a 'movies' pandas dataframe. Make sure you name the columns, use the correct separator and define the index.

In [ ]:
movies = pd.read_table('movielens/movies.dat', sep='::', names= ['ITEMID', 'Title', 'Genres'], index_col= 'ITEMID')

In [ ]:
movies.head()

Import 'ratings.dat' to a 'ratings' pandas dataframe. Make sure you name the columns, use the correct separator.

In [ ]:
ratings = pd.read_table('movielens/ratings.dat', sep='::', names= ['UserID','MovieID','Rating','Timestamp'])

In [ ]:
ratings.head()

Initialize an SVD instance

In [ ]:
svd = SVD()

Populate it with the data from the ratings dataset, using the built in load_data method

In [ ]:
svd.load_data(filename='./movielens/ratings.dat', sep='::', format={'col':0, 'row':1, 'value':2, 'ids': int})

Compute SVD

$M=U \Sigma V^T$:

In [ ]:
k = 100
svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True)

you can also save the output SVD model (in a zip file)

In [ ]:
# svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True, savefile='/tmp/movielens')

Reload a saved model:

In [ ]:
# svd2 = SVD(filename='/tmp/movielens')

Find the ITEMID number for "Toy Story (1995)"

In [ ]:
movies[movies.Title == "Toy Story (1995)"]

Find the ITEMID number for "Bug's Life, A (1998)"

In [ ]:
movies[movies.Title == "Bug's Life, A (1998)"]

Compute similarity between the two movies

In [ ]:
ITEMID1 = 1    # Toy Story (1995)
ITEMID2 = 2355 # A bug's life (1998)
print svd.similarity(ITEMID1, ITEMID2)
# print svd2.similarity(ITEMID1, ITEMID2) to check

Get movies similar to Toy Story

In [ ]:
svd.similar(ITEMID1)

Predict rating for a given user and movie, $\hat{r}_{ui}$

In [ ]:
MIN_RATING = 0.0
MAX_RATING = 5.0
ITEMID = 1
USERID = 1
svd.predict(ITEMID, USERID, MIN_RATING, MAX_RATING)

In [ ]:
svd.get_matrix().value(ITEMID, USERID)

Recommend non rated movies to a user

In [ ]:
svd.recommend(USERID, is_row=False)

Which users should see Toy Story? (e.g. which users -that have not rated Toy Story- would give it a high rating?)

In [ ]:
svd.recommend(ITEMID)

Find out more here: [https://github.com/ocelma/python-recsys](https://github.com/ocelma/python-recsys)